In [1]:
#NLP
import spacy
nlp = spacy.load("es_core_news_lg")
print(spacy.__version__)
from spacy.lang.es.stop_words import STOP_WORDS
from spacy.lang.es import Spanish
import string

#SKLEARN
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn import metrics
from sklearn.linear_model import LogisticRegression # Regresion Logística

#PANDAS
import pandas as pd

#NUMPY
import numpy as np

#PYTORCH
import torch
import random
import torchtext
from torchtext import data
from torchtext import datasets
from torchtext.legacy import data

3.1.3


# Comparacion entre modelos random forest vs cnn para la clasificacion de topicos

## Carga de datos 

In [2]:
df = pd.read_csv('datos/sophia2_data_23112021.csv')

In [3]:
df

,Unnamed: 0,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text,Dominant_Topic_Name,Dominant_Topic_REF
0,0,0,6.0,0.9859,"partido, equipo, jugar, jugador, club, chileno...",queda cerca de un mes para que comience el tor...,Deporte,7
1,1,1,14.0,0.9700,"región, caso, comuna, salud, sanitario, cuaren...",el gobierno anunció cambios en comunas de todo...,Salud,10
2,2,2,5.0,0.4317,"millón, empresa, económico, mes, mercado, dóla...","con un promedio de un 25% de desempleo, latino...",Economia,2
3,3,3,10.0,0.3685,"agua, zona, proyecto, vehículo, ambiental, exp...",¿los aficionados al queso pueden seguir disfru...,Ecologia y Planeta,8
4,4,4,15.0,0.9380,"vacuna, salud, dosis, vacunación, médico, estu...",el ministerio de salud aseguró este jueves que...,Salud,10
...,...,...,...,...,...,...,...,...
50465,50465,50465,1.0,0.9439,"carabinero, encontrar, vehículo, policía, homb...","durante la tarde de este martes, efectivos de ...","Crimen, Delitos y Justicia",9
50466,50466,50466,1.0,0.5740,"carabinero, encontrar, vehículo, policía, homb...",el primer tribunal de juicio oral en lo penal ...,"Crimen, Delitos y Justicia",9
50467,50467,50467,14.0,0.9956,"región, caso, comuna, salud, sanitario, cuaren...",este viernes las autoridades de salud entregar...,Salud,10
50468,50468,50468,15.0,0.3845,"vacuna, salud, dosis, vacunación, médico, estu...","el ministro de salud, enrique paris, reproch...",Salud,10


In [4]:
from pandasql import sqldf

In [5]:
query_sql="SELECT Dominant_Topic_Name, count(*) FROM df GROUP BY Dominant_Topic_Name ORDER BY count(*) DESC;"
df_result=sqldf(query_sql)
df_result

,Dominant_Topic_Name,count(*)
0,Salud,10449
1,"Crimen, Delitos y Justicia",9354
2,Politica y Conflictos,8861
3,Cultura y Artes,6567
4,Deporte,5078
5,Mundo,3287
6,Economia,2755
7,Ecologia y Planeta,2640
8,Catastrofes y Accidentes,1479


In [6]:
X = df['Text'] # the features we want to analyze
ylabels = df['Dominant_Topic_Name'] # the labels, or answers, we want to test against

X_train, X_test, y_train, y_test = train_test_split(X, ylabels, test_size=0.25)

In [7]:
# Create our list of punctuation marks
punctuations = [".",",","!","?", "#","&"]

# Create our list of stopwords
stop_words=[""]

# Load English tokenizer, tagger, parser, NER and word vectors
parser = Spanish()

# Creating our tokenizer function
def spacy_tokenizer(sentence):
    # Creating our token object, which is used to create documents with linguistic annotations.
    mytokens = parser(sentence)

    # Lemmatizing each token and converting each token into lowercase
    mytokens = [word.lower_ for word in mytokens]
        
    # Removing stop words
    mytokens = [ word for word in mytokens if word not in stop_words and word not in punctuations ]

    # return preprocessed list of tokens
    return mytokens

In [8]:
bow_vector = CountVectorizer(tokenizer = spacy_tokenizer, ngram_range=(1,1))

CountVectorizer(tokenizer=<function spacy_tokenizer at 0x000001FC93166D30>)

## Modelo Random Forest

In [9]:
from sklearn.ensemble import RandomForestClassifier
modelRF = RandomForestClassifier(random_state=0)

model3 = Pipeline([('preprocessing', bow_vector),
                 ('regression-ML', modelRF)])

# model generation
model3.fit(X_train, y_train)

Pipeline(steps=[('preprocessing',
                 CountVectorizer(tokenizer=<function spacy_tokenizer at 0x000001FC93166D30>)),
                ('regression-ML', RandomForestClassifier(random_state=0))])

In [10]:
# Predicting with a test dataset
predicted = model3.predict(X_test)
print(predicted)

# Model Accuracy
print("Logistic Regression Accuracy:", metrics.accuracy_score(y_test, predicted))

['Salud' 'Salud' 'Crimen, Delitos y Justicia' ... 'Deporte' 'Deporte'
 'Politica y Conflictos']
Logistic Regression Accuracy: 0.823585354255825


### Diagnosticos

In [11]:
#Evaluación del rendimiento del clasificador
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(y_test, predicted)
print(confusion_matrix)

#Print de la matriz de confusión
from sklearn.metrics import classification_report
print(classification_report(y_test, predicted))

[[ 157  118    9    0    0    1    1    5   79]
 [   1 2058    6    4    0    1   10  107   61]
 [   1   81 1386   56    4    0    9   37   84]
 [   0    1   12 1216    0    1    1    4    6]
 [   4   98   74    5  259    6   15   36  189]
 [   0   68   25    7    2  284   14  115  202]
 [   0  106   22    2    0    0  538   95   68]
 [   0   63    4   11    0    2   31 2033  107]
 [   0   56   20   10    1    3    7   58 2461]]
                            precision    recall  f1-score   support

  Catastrofes y Accidentes       0.96      0.42      0.59       370
Crimen, Delitos y Justicia       0.78      0.92      0.84      2248
           Cultura y Artes       0.89      0.84      0.86      1658
                   Deporte       0.93      0.98      0.95      1241
        Ecologia y Planeta       0.97      0.38      0.54       686
                  Economia       0.95      0.40      0.56       717
                     Mundo       0.86      0.65      0.74       831
     Politica y Confli

## Modelo CNN

In [8]:
TEXT = data.Field(tokenize=spacy_tokenizer, batch_first = True)
CATEGORY = data.LabelField()

In [9]:
valid, test, train = np.split(df, [int(.15*len(df)), int(.3*len(df))])

In [11]:
fields = [(None, None),(None, None),(None, None),(None, None),(None, None),('body', TEXT),('category', CATEGORY), (None, None)]

In [12]:
train.to_csv("datos/sohpia2_train.csv", encoding="UTF-8",index=False)
valid.to_csv("datos/sophia2_valid.csv", encoding="UTF-8",index=False)
test.to_csv("datos/sophia2_test.csv", encoding="UTF-8",index=False)

In [13]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

train_data, valid_data, test_data = data.TabularDataset.splits(
                                        path = '.',
                                        train = 'datos/sohpia2_train.csv',
                                        validation= 'datos/sophia2_valid.csv',
                                        test = 'datos/sophia2_test.csv',
                                        format = 'csv',
                                        fields = fields,
                                        skip_header = True
)

In [14]:
BATCH_SIZE = 32

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE, 
    device = device,
    sort_key=lambda x:len(x.category),
    sort_within_batch=False)

cuda


In [15]:
MAX_VOCAB_SIZE = 50000

## TENER VECTORES EN ESPAÑOL
vec = torchtext.vocab.Vectors('glove-sbwc.i25.vec.gz', cache='.')
TEXT.build_vocab(train_data, vectors=vec, max_size = MAX_VOCAB_SIZE, unk_init = torch.Tensor.normal_)

CATEGORY.build_vocab(train_data)

In [16]:
print(CATEGORY.vocab.stoi)

defaultdict(None, {'Salud': 0, 'Crimen, Delitos y Justicia': 1, 'Politica y Conflictos': 2, 'Cultura y Artes': 3, 'Deporte': 4, 'Mundo': 5, 'Ecologia y Planeta': 6, 'Economia': 7, 'Catastrofes y Accidentes': 8})


In [17]:
import torch.nn as nn
import torch.nn.functional as F

class CNN1d(nn.Module):
    def __init__(self, vocab_size, embedding_dim, n_filters, filter_sizes, output_dim, 
                 dropout, pad_idx):
        
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)
        
        self.convs = nn.ModuleList([
                                    nn.Conv1d(in_channels = embedding_dim, 
                                              out_channels = n_filters, 
                                              kernel_size = fs)
                                    for fs in filter_sizes
                                    ])
        
        self.fc = nn.Linear(len(filter_sizes) * n_filters, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):
        
        embedded = self.embedding(text)
                
        embedded = embedded.permute(0, 2, 1)
        
        conved = [F.relu(conv(embedded)) for conv in self.convs]

        pooled = [F.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in conved]
        
        cat = self.dropout(torch.cat(pooled, dim = 1))
        
        return self.fc(cat)

In [18]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 300
N_FILTERS = 100
FILTER_SIZES = [3]
OUTPUT_DIM = len(CATEGORY.vocab)
DROPOUT = 0.5
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = CNN1d(INPUT_DIM, EMBEDDING_DIM, N_FILTERS, FILTER_SIZES, OUTPUT_DIM, DROPOUT, PAD_IDX)

In [19]:
OUTPUT_DIM

9

In [20]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 15,091,609 trainable parameters


In [21]:
pretrained_embeddings = TEXT.vocab.vectors
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

model.embedding.weight.data.copy_(pretrained_embeddings)
model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

In [22]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())

criterion = nn.CrossEntropyLoss()

model = model.to(device)
criterion = criterion.to(device)

In [23]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        
        predictions = model(batch.body).squeeze(1)
        
        loss = criterion(predictions, batch.category)
        
        acc = categorical_accuracy(predictions, batch.category)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [24]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [25]:
def categorical_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """
    max_preds = preds.argmax(dim = 1, keepdim = True) # get the index of the max probability
    correct = max_preds.squeeze(1).eq(y)
    return correct.sum() / torch.cuda.FloatTensor([y.shape[0]])

In [26]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:
           
            predictions = model(batch.body).squeeze(1)
            
            loss = criterion(predictions, batch.category)
            
            acc = categorical_accuracy(predictions, batch.category)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
print("inicio optimización")

N_EPOCHS = 20

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        nombre = './models/topic-model-sophia2'+'_ep'+str(epoch+1)+'.pt'
        torch.save({'epoca': epoch,
                    'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'Valid_loss': best_valid_loss}, nombre)
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')


In [35]:
test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.915 | Test Acc: 86.29%


### Diagnosticos

In [91]:
from sklearn.metrics import confusion_matrix

nb_classes = 9

# Initialize the prediction and label lists(tensors)
predlist=torch.zeros(0,dtype=torch.long, device='cpu')
lbllist=torch.zeros(0,dtype=torch.long, device='cpu')

device = 'cpu'
model.to(device)

with torch.no_grad():
    for i, (inputs, classes) in enumerate(valid_iterator):
        inputs = inputs.to(device)
        classes = classes.to(device)
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)

        # Append batch prediction results
        predlist=torch.cat([predlist,preds.view(-1).cpu()])
        lbllist=torch.cat([lbllist,classes.view(-1).cpu()])

# Confusion matrix
conf_mat=confusion_matrix(lbllist.numpy(), predlist.numpy())
print(conf_mat)

# Per-class accuracy
class_accuracy=100*conf_mat.diagonal()/conf_mat.sum(1)
print(class_accuracy)

# Classification report
from sklearn.metrics import classification_report
print(classification_report(lbllist.numpy(), predlist.numpy()))

[[1419   33   49   19    3   21   17   24    5]
 [  33 1240   53   24    3   32   14    4   19]
 [  41   51 1177   14    5   30   16   11    1]
 [  23   23   12  845   18   13   18    7    3]
 [   3    2    0   21  723    0    3    2    0]
 [  17   23   17   11    1  396    6    3    1]
 [  18    7    5   15    2    6  301    6    8]
 [  28   16   25   16    3   14   13  328    1]
 [  14   23    1    2    2    0   18    1  148]]
[89.24528302 87.20112518 87.44427935 87.83783784 95.88859416 83.36842105
 81.79347826 73.87387387 70.81339713]
              precision    recall  f1-score   support

           0       0.89      0.89      0.89      1590
           1       0.87      0.87      0.87      1422
           2       0.88      0.87      0.88      1346
           3       0.87      0.88      0.88       962
           4       0.95      0.96      0.96       754
           5       0.77      0.83      0.80       475
           6       0.74      0.82      0.78       368
           7       0.85